# Lung Cancer Classification using Computerized Tomography
 Project developed by: **Eduardo Passos** [202205630](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202205630), **Pedro Fernandes** [202208347](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202208347) and **Rafael Pacheco** [202206258](https://sigarra.up.pt/fcup/pt/fest_geral.cursos_list?pv_num_unico=202206258)


### Index {#index} #############################################
1. [Project Introduction](#intro)
2. [Data Understanding](#understand)
3. [Pre-Processing and Conversion](#preproc)
4. [Bagging for Features](#bagging)
5. [Exploratory Data Analysis](#eda)
6. [Models](#models)
6.1. [Support Vector Machines](#svm)


## 1. Project Introduction and Motivation {#intro}
[Back to Index](#index)

This project focuses on classifying lung cancer using Computed Tomography (CT) scan data, using the [**LIDC-IDRI**](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset. 

As referenced in the assignment, **lung cancer leads global cancer-related mortality rates**, with only 16% of cases detected at a localized stage. In these instances, patients have a five-year survival rate exceeding 50%. However, when diagnosed at an advanced stage, the five-year survival rate drops to just 5%. Therefore, **early diagnosis is crucial for improving survival outcomes**, and systems designed to assist with screening can be instrumental regarding this matter (statistics from [World Health Organization](https://www.iarc.who.int/featured-news/latest-global-cancer-data-cancer-burden-rises-to-18-1-million-new-cases-and-9-6-million-cancer-deaths-in-2018/) and [American Cancer Society](https://www.cancer.org/research/cancer-facts-statistics/all-cancer-facts-figures/cancer-facts-figures-2019.html)).

The goal is to **develop a machine learning model capable of accurately identifying lung cancer** (based on the provided medical imaging data) in early stages of the tumour, in order to prevent evolution of the disease. To support this, we have developed a program that **converts the dataset** (initially in DICOM format) into numpy array files (.npy), while extracting and storing key information in a CSV file. The preprocessed data will be utilized to **train the ??????????????????????????**, enabling accurate classification of the nodules from the imaging data.

## 2. Data Understanding {#understand}
[Back to Index](#index)

The [LIDC-IDRI](https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=1966254) dataset **contains the imaging data of the lungs** from over 1000 patients. This dataset is part of a collection of datasets provided by [**The Cancer Imaging Archive**](https://www.cancerimagingarchive.net/browse-collections/) for public research. The medical imaging data is provided in [DICOM](https://en.wikipedia.org/wiki/DICOM) (.dcm), and contains both image data, and associated metadata. It also contains the annotations of the medical professionals regarding the subject. 

The nodule annotations in the dataset **reflect the radiologist's evaluation** and include key information such as nodule size and malignancy rating (on a scale from 1-5). However, the **malignancy classification may vary based on each radiologist's judgment** and reasoning. What this means is that it introduces a level of subjectivity, making the classification potentially ambiguous and not always straightforward.

The dataset has also some noise, missing data and overall file corruption which has to be dealt with. Some patients do not have annotations, some patients have no nodule at all, and some nodules have missing information.

To help with the learning process, the values of Hounsfield Units (HU) as pixel values were normalized. (????????????????? completar)

(???)

## 3. Pre-Processing and Conversion {#preproc}
[Back to Index](#index)

Resumir o codigo do steps essencialmente, tudo o que foi pre processado e criado e como, augmentation, dps o codigo ha de entrar na pasta.


## 4. Exploratory Data Analysis {#eda}
[Back to Index](#index)

